## Section 1: Setup & Imports
Install and import libraries for semantic analysis, visualization, parsing, and code generation.

In [ ]:
# Install dependencies
!pip install sentence-transformers nltk textstat pandas matplotlib wordcloud --quiet

# Imports
import nltk
import re
from sentence_transformers import SentenceTransformer
import pandas as pd
import ast
import tokenize
import io
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Download NLTK data, including punkt_tab
try:
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading NLTK data: {e}")
print("[✓] Setup complete")

[✓] Setup complete


## Section 2: Example Code Snippets

In [ ]:
code_snippets = [
    """# 1. Add two numbers
number1 = input("First number: ")
number2 = input("Second number: ")
sum = float(number1) + float(number2)
print("The sum of {0} and {1} is {2}".format(number1, number2, sum))
""",
    """# 2. Factorial of a number
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)
num = 5
print("Factorial of", num, "is", factorial(num))
""",
    """# 3. Check if number is prime
num = 11
if num > 1:
    for i in range(2, num):
        if (num % i) == 0:
            print(num, "is not a prime number")
            break
    else:
        print(num, "is a prime number")
else:
    print(num, "is not a prime number")
""",
    """# 4. Find area of a circle
radius = 6
pi = 3.14159
area = pi * radius * radius
print("Area of circle is:", area)
""",
    """# 5. Fibonacci sequence
def fibonacci(n):
    if n <= 0:
        print("Incorrect input")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
print(fibonacci(9))
""",
    """# 6. Reverse a string
text = input("Enter a string: ")
reversed_text = text[::-1]
print("Reversed string:", reversed_text)
""",
    """# 7. Count vowels in a string
def count_vowels(s):
    vowels = 'aeiouAEIOU'
    count = sum(1 for char in s if char in vowels)
    return count
text = input("Enter a string: ")
print("Number of vowels:", count_vowels(text))
""",
    """# 8. Sum of list elements
numbers = [1, 2, 3, 4, 5]
total = sum(numbers)
print("Sum of list:", total)
"""
]

snippet_names = [f"Program_{i+1}" for i in range(len(code_snippets))]

print("Loaded 8 example code snippets:")
for name, text in zip(snippet_names, code_snippets):
    print(f"{name}: {len(text)} characters")


Loaded 8 example code snippets:
Program_1: 197 characters
Program_2: 188 characters
Program_3: 268 characters
Program_4: 113 characters
Program_5: 235 characters
Program_6: 123 characters
Program_7: 225 characters
Program_8: 102 characters


## Section 3: Multi-Model Code Explainer Module
Use three pretrained SentenceTransformer models to generate context-aware explanations.
Compare explanations across models.

In [ ]:
class CodeExplainer:
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(snippet_names, code_snippets):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
print(comparisons[0][1].to_string(index=False))

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        print(differing_lines.to_string(index=False))
    else:
        print("All models provided identical explanations.")



Explanation Comparison for Program_1:
 Line Number                                                                 Code                                                                          MiniLM                                                                               DistilRoBERTa                                                                                 MPNet
           2                                    number1 = input("First number: ")                                                      Assigns value to 'number1'                                                          Sets variable 'number1' to a value                                                            Stores result in 'number1'
           3                                   number2 = input("Second number: ")                                                      Assigns value to 'number2'                                                          Sets variable 'number2' to a value                            

# Task
Implement a Python interface `codeexplainerinterface` with methods `explain_code` and `compare_explanations`, and modify the existing `codeexplainer` class to implement this interface.

## Define the `codeexplainerinterface`

### Subtask:
Create an abstract base class (ABC) in Python to define the interface with the required methods (`explain_code` and `compare_explanations`).


**Reasoning**:
Define the abstract base class `CodeExplainerInterface` with the specified abstract methods as requested by the subtask.



In [ ]:
from abc import ABC, abstractmethod

class CodeExplainerInterface(ABC):
    @abstractmethod
    def explain_code(self, code_text, model_name):
        """
        Abstract method to explain code using a specific model.

        Args:
            code_text: The code snippet as a string.
            model_name: The name of the model to use for explanation.

        Returns:
            A dictionary of explanations, where keys are line numbers and values are explanations.
        """
        pass

    @abstractmethod
    def compare_explanations(self, code_text, snippet_name):
        """
        Abstract method to compare explanations from multiple models for a code snippet.

        Args:
            code_text: The code snippet as a string.
            snippet_name: The name of the code snippet.

        Returns:
            A pandas DataFrame comparing explanations from different models.
        """
        pass

## Modify `codeexplainer` to implement the interface

### Subtask:
Update the existing `CodeExplainer` class to inherit from the `CodeExplainerInterface` and ensure it implements all the required methods.


**Reasoning**:
Modify the existing CodeExplainer class to inherit from CodeExplainerInterface and keep the existing methods.



In [ ]:
class CodeExplainer(CodeExplainerInterface):
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(snippet_names, code_snippets):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
display(comparisons[0][1])

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        display(differing_lines)
    else:
        print("All models provided identical explanations.")

NameError: name 'SentenceTransformer' is not defined

**Reasoning**:
The previous code block failed because SentenceTransformer was not defined. Need to include the import statement for SentenceTransformer.



In [ ]:
from sentence_transformers import SentenceTransformer

class CodeExplainer(CodeExplainerInterface):
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(snippet_names, code_snippets):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
display(comparisons[0][1])

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        display(differing_lines)
    else:
        print("All models provided identical explanations.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'snippet_names' is not defined

**Reasoning**:
The previous code block failed because `snippet_names` and `code_snippets` were not defined in the current cell. I need to include the definition of these variables from the previous relevant cell.



In [ ]:
from sentence_transformers import SentenceTransformer

code_snippets = [
    """# 1. Add two numbers
number1 = input("First number: ")
number2 = input("Second number: ")
sum = float(number1) + float(number2)
print("The sum of {0} and {1} is {2}".format(number1, number2, sum))
""",
    """# 2. Factorial of a number
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)
num = 5
print("Factorial of", num, "is", factorial(num))
""",
    """# 3. Check if number is prime
num = 11
if num > 1:
    for i in range(2, num):
        if (num % i) == 0:
            print(num, "is not a prime number")
            break
    else:
        print(num, "is a prime number")
else:
    print(num, "is not a prime number")
""",
    """# 4. Find area of a circle
radius = 6
pi = 3.14159
area = pi * radius * radius
print("Area of circle is:", area)
""",
    """# 5. Fibonacci sequence
def fibonacci(n):
    if n <= 0:
        print("Incorrect input")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
print(fibonacci(9))
""",
    """# 6. Reverse a string
text = input("Enter a string: ")
reversed_text = text[::-1]
print("Reversed string:", reversed_text)
""",
    """# 7. Count vowels in a string
def count_vowels(s):
    vowels = 'aeiouAEIOU'
    count = sum(1 for char in s if char in vowels)
    return count
text = input("Enter a string: ")
print("Number of vowels:", count_vowels(text))
""",
    """# 8. Sum of list elements
numbers = [1, 2, 3, 4, 5]
total = sum(numbers)
print("Sum of list:", total)
"""
]

snippet_names = [f"Program_{i+1}" for i in range(len(code_snippets))]


class CodeExplainer(CodeExplainerInterface):
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(snippet_names, code_snippets):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
display(comparisons[0][1])

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        display(differing_lines)
    else:
        print("All models provided identical explanations.")

NameError: name 'pd' is not defined

**Reasoning**:
The previous code block failed because `pd` was not defined. I need to include the import statement for pandas.



In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

code_snippets = [
    """# 1. Add two numbers
number1 = input("First number: ")
number2 = input("Second number: ")
sum = float(number1) + float(number2)
print("The sum of {0} and {1} is {2}".format(number1, number2, sum))
""",
    """# 2. Factorial of a number
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)
num = 5
print("Factorial of", num, "is", factorial(num))
""",
    """# 3. Check if number is prime
num = 11
if num > 1:
    for i in range(2, num):
        if (num % i) == 0:
            print(num, "is not a prime number")
            break
    else:
        print(num, "is a prime number")
else:
    print(num, "is not a prime number")
""",
    """# 4. Find area of a circle
radius = 6
pi = 3.14159
area = pi * radius * radius
print("Area of circle is:", area)
""",
    """# 5. Fibonacci sequence
def fibonacci(n):
    if n <= 0:
        print("Incorrect input")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
print(fibonacci(9))
""",
    """# 6. Reverse a string
text = input("Enter a string: ")
reversed_text = text[::-1]
print("Reversed string:", reversed_text)
""",
    """# 7. Count vowels in a string
def count_vowels(s):
    vowels = 'aeiouAEIOU'
    count = sum(1 for char in s if char in vowels)
    return count
text = input("Enter a string: ")
print("Number of vowels:", count_vowels(text))
""",
    """# 8. Sum of list elements
numbers = [1, 2, 3, 4, 5]
total = sum(numbers)
print("Sum of list:", total)
"""
]

snippet_names = [f"Program_{i+1}" for i in range(len(code_snippets))]


class CodeExplainer(CodeExplainerInterface):
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(snippet_names, code_snippets):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
display(comparisons[0][1])

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        display(differing_lines)
    else:
        print("All models provided identical explanations.")


Explanation Comparison for Program_1:


,Line Number,Code,MiniLM,DistilRoBERTa,MPNet
0,2,"number1 = input(""First number: "")",Assigns value to 'number1',Sets variable 'number1' to a value,Stores result in 'number1'
1,3,"number2 = input(""Second number: "")",Assigns value to 'number2',Sets variable 'number2' to a value,Stores result in 'number2'
2,4,sum = float(number1) + float(number2),Assigns value to 'sum',Sets variable 'sum' to a value,Stores result in 'sum'
3,5,"print(""The sum of {0} and {1} is {2}"".format(n...","Outputs: 'print(""The sum of {0} and {1} is {2}...","Displays to console: 'print(""The sum of {0} an...","Prints result: 'print(""The sum of {0} and {1} ..."



Summary of Explanation Differences:

Program_1:
Lines with differing explanations:


,Line Number,Code,MiniLM,DistilRoBERTa,MPNet
0,2,"number1 = input(""First number: "")",Assigns value to 'number1',Sets variable 'number1' to a value,Stores result in 'number1'
1,3,"number2 = input(""Second number: "")",Assigns value to 'number2',Sets variable 'number2' to a value,Stores result in 'number2'
2,4,sum = float(number1) + float(number2),Assigns value to 'sum',Sets variable 'sum' to a value,Stores result in 'sum'
3,5,"print(""The sum of {0} and {1} is {2}"".format(n...","Outputs: 'print(""The sum of {0} and {1} is {2}...","Displays to console: 'print(""The sum of {0} an...","Prints result: 'print(""The sum of {0} and {1} ..."



Program_2:
Lines with differing explanations:


...

,Line Number,Code,MiniLM,DistilRoBERTa,MPNet
0,2,"text = input(""Enter a string: "")",Assigns value to 'text',Sets variable 'text' to a value,Stores result in 'text'
1,3,reversed_text = text[::-1],Assigns value to 'reversed_text',Sets variable 'reversed_text' to a value,Stores result in 'reversed_text'
2,4,"print(""Reversed string:"", reversed_text)","Outputs: 'print(""Reversed string:"", reversed_t...","Displays to console: 'print(""Reversed string:""...","Prints result: 'print(""Reversed string:"", reve..."



Program_7:
Lines with differing explanations:


,Line Number,Code,MiniLM,DistilRoBERTa,MPNet
0,2,def count_vowels(s):,Defines function 'count_vowels' with parameters,Creates a function 'count_vowels' for computation,Sets up function 'count_vowels' to perform a task
1,3,vowels = 'aeiouAEIOU',Assigns value to 'vowels',Sets variable 'vowels' to a value,Stores result in 'vowels'
2,4,count = sum(1 for char in s if char in vowels),Assigns value to 'count',Sets variable 'count' to a value,Stores result in 'count'
3,5,return count,Returns: 'return count',Yields value: 'return count',Provides output: 'return count'
4,6,"text = input(""Enter a string: "")",Assigns value to 'text',Sets variable 'text' to a value,Stores result in 'text'
5,7,"print(""Number of vowels:"", count_vowels(text))","Outputs: 'print(""Number of vowels:"", count_vow...","Displays to console: 'print(""Number of vowels:...","Prints result: 'print(""Number of vowels:"", cou..."



Program_8:
Lines with differing explanations:


,Line Number,Code,MiniLM,DistilRoBERTa,MPNet
0,2,"numbers = [1, 2, 3, 4, 5]",Assigns value to 'numbers',Sets variable 'numbers' to a value,Stores result in 'numbers'
1,3,total = sum(numbers),Assigns value to 'total',Sets variable 'total' to a value,Stores result in 'total'
2,4,"print(""Sum of list:"", total)","Outputs: 'print(""Sum of list:"", total)'","Displays to console: 'print(""Sum of list:"", to...","Prints result: 'print(""Sum of list:"", total)'"


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=differing_lines)

## Demonstrate the interface usage

### Subtask:
Write a code snippet that shows how to use the `CodeExplainerInterface` to interact with the `CodeExplainer` class.


**Reasoning**:
Instantiate a CodeExplainer object, define a sample code snippet, and use the explain_code and compare_explanations methods as specified in the instructions.



In [ ]:
# 1. Instantiate an object of the CodeExplainer class.
explainer = CodeExplainer()

# 2. Define a sample code snippet string.
sample_code = """
def greet(name):
    print(f"Hello, {name}!")

greet("World")
"""

# 3. Use the explain_code method and print the explanations.
print("Explanation using MiniLM:")
explanations_minilm = explainer.explain_code(sample_code, 'MiniLM')
for line_num, explanation in explanations_minilm.items():
    print(f"Line {line_num}: {explanation}")

# 4. Use the compare_explanations method and display the DataFrame.
print("\nComparison of Explanations for Sample Program:")
comparison_df = explainer.compare_explanations(sample_code, 'Sample Program')
display(comparison_df)

Explanation using MiniLM:
Line 2: Defines function 'greet' with parameters
Line 3: Outputs: 'print(f"Hello, {name}!")'
Line 5: Executes: 'greet("World")'

Comparison of Explanations for Sample Program:


,Line Number,Code,MiniLM,DistilRoBERTa,MPNet
0,2,def greet(name):,Defines function 'greet' with parameters,Creates a function 'greet' for computation,Sets up function 'greet' to perform a task
1,3,"print(f""Hello, {name}!"")","Outputs: 'print(f""Hello, {name}!"")'","Displays to console: 'print(f""Hello, {name}!"")'","Prints result: 'print(f""Hello, {name}!"")'"
2,5,"greet(""World"")","Executes: 'greet(""World"")'","Performs operation: 'greet(""World"")'","Runs code: 'greet(""World"")'"


from matplotlib import pyplot as plt
comparison_df['Line Number'].plot(kind='hist', bins=20, title='Line Number')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
comparison_df.groupby('Code').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
comparison_df.groupby('MiniLM').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(comparison_df['MiniLM'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(comparison_df, x='Line Number', y='MiniLM', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(comparison_df['DistilRoBERTa'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(comparison_df, x='Line Number', y='DistilRoBERTa', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(comparison_df['MPNet'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(comparison_df, x='Line Number', y='MPNet', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=comparison_df)

## Summary:

### Data Analysis Key Findings

*   An abstract base class `CodeExplainerInterface` was successfully created with abstract methods `explain_code` and `compare_explanations`.
*   The `CodeExplainer` class was modified to inherit from `CodeExplainerInterface` and implement its methods.
*   The implementation of `CodeExplainer` includes using `SentenceTransformer` models ('MiniLM', 'DistilRoBERTa', 'MPNet') and predefined explanation templates to generate code explanations.
*   The `explain_code` method provides line-by-line explanations for a given code snippet using a specified model.
*   The `compare_explanations` method generates a pandas DataFrame comparing the explanations from all available models for a code snippet.
*   A demonstration snippet successfully showed how to instantiate `CodeExplainer` and use both `explain_code` and `compare_explanations` methods, displaying the results.

### Insights or Next Steps

*   The current explanation generation is based on simple pattern matching and predefined templates. A next step could involve integrating more sophisticated natural language generation techniques based on the semantic embeddings produced by the SentenceTransformer models for more nuanced explanations.
*   The comparison of explanations highlights differences between models. Further analysis could involve evaluating the quality and accuracy of explanations from each model against a human-annotated ground truth to determine which model performs best for different types of code constructs.
